In [1]:
import requests
import hashlib
from datetime import datetime
import time
import random
import os
import csv
from bs4 import BeautifulSoup
import re 
import pandas as pd

/Users/pandabear/anaconda/lib/python3.6/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


# Parsing functions

In [208]:
def parse_listing_html(text,file_name):
    #define the html text
    page_html = BeautifulSoup(text, 'html.parser')
    
    if deleted_listing(page_html):
        print('listing {} has been deleted'.format(file_name))
        return
    
    listing_title = page_html.find('span',attrs={'id':'titletextonly'}).text     
    listing_date = page_html.find('time', class_='date timeago')['datetime']
    listing_price = page_html.find('span', class_='price')

    listing_bedbath= page_html.find('span', class_='shared-line-bubble').text

    # Extract sqft info from bubbles class and use regex to find the digits ending in ft2
    bubbles_sqft = page_html.findAll('span', class_='shared-line-bubble')
    listing_sqft = None
    if len(bubbles_sqft) >= 2:
        x = re.search("[0-9]+ft2$", bubbles_sqft[1].text)
        if x:
             listing_sqft = x.string[:-3]
    
    listing_address_element = page_html.find('div', class_='mapaddress')
    listing_address = listing_address_element and listing_address_element.text

    listing_info = page_html.findAll('p', class_='attrgroup')[1].text

    listing_body = page_html.find('section',attrs={'id':'postingbody'}).text
    
    listing_id = page_html.findAll('p', class_='postinginfo')[1].text
    
    listing_url = page_html.find('meta',attrs={'property':'og:url'}).attrs['content']
    
    listing_first_image = page_html.find('div',class_='slide first visible').img['src']


    return {
        'listing_title': listing_title,
        'listing_date':listing_date,
        'listing_price': int(listing_price.text.strip('$,').replace(",","")),
        'listing_bedbath':listing_bedbath,
        'listing_sqft':listing_sqft,
        'listing_address':listing_address,
        'listing_info':listing_info,
        'listing_body':listing_body,
        'listing_id':listing_id,
        'listing_url':listing_url,
        'listing_first_image':listing_first_image
       }


def deleted_listing(page_html):
    h2 = page_html.find('h2')
    return h2 and h2.text == ' This posting has been deleted by its author. '

# Create df from dictionary

In [232]:
list_of_dicts = []
path = "/Users/pandabear/springboard/CapstoneTwoProject/data/raw"
dir_list = os.listdir(path)
#print(dir_list)
for folder in dir_list:
    if not folder.startswith('.'):
        path_html = path + '/' + folder
        file_list = os.listdir(path_html)
        #print(file_list)
        
        for file in file_list:
            if len(file) > 15:
                with open(path_html + '/' + file, 'r') as f:
                    text = f.read()
                    parsed_data = parse_listing_html(text, file)
                    if parsed_data:
                        list_of_dicts.append(parsed_data)
#                 print('Number of parsing errors: {}'.format(count_parsing_error))
        listing_df = pd.DataFrame(list_of_dicts)


listing b3683c62f193f873f9aba874e4a128dd21cebc8490b6efe86004f613d66a3d47 has been deleted
listing 046d614c116efc3eb96aa3e0ab1834f39f073ae45c7e3a2746b65133f25546bd has been deleted
listing 8eed6b17707dd43e898e6afbd7d279f9f3e845f26a4c3b679b6f8aa849e43dd5 has been deleted
listing e85010522f800de77e8fd82549ff44cc01d9ab5b78799e2ee429fc0fb18a4ab0 has been deleted
listing f4b208fc87aa63d1425261c99c7dbb22c38d849ed7f3a12f928403f9c55ae02d has been deleted
listing 1da2569bead691e42984de0953e63b74d55773d9654af42924ed248a928513b9 has been deleted
listing ebfd9113e3ceacc6ab51eca901a409a932b1ecaa3fbcd5dff73c7b06e00f795b has been deleted
listing 41d856cda9abf5df2ad14c49f615b5a18374e1a370b6fdd0a0fe3c32994e61b6 has been deleted
listing 5c9a78f9c20ae922e6008c0cc89bb2b8e85939c90fcbbd47517174dc02c775c0 has been deleted
listing 5064888afec96047ab8afe015fdd9047d0360b529bdf57cfb3b9b656f8725bae has been deleted
listing c92b9fbe52321e85a68d4ea79dacabdd7262878a1f8b0c363220709c42401b8a has been deleted
listing 2f

In [241]:
listing_df.set_index('listing_id', inplace = True)
listing_df.head()

,listing_address,listing_bedbath,listing_body,listing_date,listing_first_image,listing_info,listing_price,listing_sqft,listing_title,listing_url
listing_id,,,,,,,,,,
post id: 7517797369,1537 Jones St near Pacific,1BR / 1Ba,QR Code Link to This Post This is a 1BR 1B...,2022-08-05T13:48:22-0700,https://images.craigslist.org/00X0X_aCkV5CnbZ2...,apartment laundry in bldg attached garage ...,2375,None,"Cozy 1BR 1BA, Clean Unit, Walk to Chinatown/Fi...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...
post id: 7519257988,1290 20th ave.,1BR / 1Ba,QR Code Link to This Post 1290 20th Ave. #...,2022-08-09T09:19:13-0700,https://images.craigslist.org/00707_34DEnZksvZ...,application fee details: $35 cats are OK - p...,2995,None,1290 20thAve.&Irving/Awesome views/Available N...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...
post id: 7515346170,1825 Mission Street near 14th street.,0BR / 1Ba,QR Code Link to This Post Welcome to 1825 ...,2022-07-30T12:42:45-0700,https://images.craigslist.org/00y0y_gIofOzLqJq...,cats are OK - purrr dogs are OK - wooof apa...,1950,274,"Beautiful Landscaping, Stainless Steel Applian...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...
post id: 7514598032,van ness near greenwich street,0BR / 1Ba,QR Code Link to This Post MORE THAN JUST A...,2022-07-28T15:10:30-0700,https://images.craigslist.org/00101_61nNL8QiJS...,cats are OK - purrr dogs are OK - wooof apa...,2850,580,Deluxe Bright Studio in The Marina Chateau,https://sfbay.craigslist.org/sfc/apa/d/san-fra...
post id: 7517784826,780 Post street,0BR / 1Ba,"QR Code Link to This Post 780 Post Street,...",2022-08-05T13:21:42-0700,https://images.craigslist.org/00M0M_cGnsXvY7FB...,cats are OK - purrr dogs are OK - wooof apa...,1575,None,Studio in Historic Pet-friendly Building! 780 ...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...


In [235]:
listing_df.shape

(6211, 11)

In [242]:
listing_df.to_csv(r'/Users/pandabear/springboard/CapstoneTwoProject/data/interim/listing_df.csv', header=True)
print('Saved to csv file')

Saved to csv file


In [ ]:
#neighborhoods in SF
S,F_neighborhoods = ['alamo square / nopa', 'bayview', 'bernal heights', 'castro / upper market', 'cole valley / ashbury hts', 'downtown / civic / van ness'
 'excelsior / outer mission', 'financial district', 'glen park', 'haight ashbury', 'hayes valley', 'ingleside / SFSU / CCSF', 'inner richmond'
 'inner sunset / UCSF', 'laurel hts / presidio', 'lower haight', 'lower nob hill', 'lower pac hts', 'marina / cow hollow', 'mission district'
 'nob hill', 'noe valley', 'north beach / telegraph hill', 'pacific heights', 'portola district', 'potrero hill', 'richmond / seacliff', 'russian hill', 'SOMA / south beach', 'sunset / parkside', 'tenderloin'
 'treasure island', 'twin peaks / diamond hts', 'USF / panhandle', 'visitacion valley', 'west portal / forest hill', 'western addition']